In [1]:
import pandas as pd
import numpy as np

# Explain the meaning of each feature

- DATASOURCE：Bus Operator Code
- DAYOFSERVICE：  Bus service date
- TRIPID： Trip ID Check if the same trip needs to be combined with DAYOFSERVICE
- LINEID： Route ID
- ROUTEID： Route ID
- DIRECTION ：Indicates the direction of the bus form (forward or reverse)
- PLANNEDTIME_ARR：  Planned trip arrival time
- PLANNEDTIME_DEP ：  Planned trip departure time
- ACTUALTIME_ARR ：  Actual trip arrival time
- ACTUALTIME_DEP：  Actual trip departure time
- BASIN ： Basin code
- TENDERLOT ：   Tender lot
- SUPPRESSED ：  Whether suppressed at this station (0 for arrival, 1 for suppressed)
- JUSTIFICATIONID ：  fault code
- LASTUPDATE ： Time of last update
- NOTE：   Note

# New feature
- HOUR: Hour
- MONTH: month
- WEEKDAY: day of the week
- RUSHHOUR: whether or not the trip is in peak traffic, peak periods are defined as 8,9,10,16,17,18, 1 if the hour is in the peak period, 0 otherwise
- JOURNEYTIME: the time of the entire journey
- STOP_NUM:Number of stops travelled on this trip
- LINE_STOP_NUM:Number of stops travelled on this line
- VG_STOP_TIME_MAX:On the basis of the number of stops on the line, the average time per stop is obtained（JOURNEYTIME/LINE_STOP_NUM）
- AVG_STOP_TIME:On the basis of the number of stops on the trip, the average time per stop is obtained（JOURNEYTIME/STOP_NUM）

### Summary of data quality plan:
| Variable Names                     | Data Quality Issue            | Handling Strategy              |
|------------------------------------|-------------------------------|--------------------------------|
| DATASOURCE  |Constant column | Drop this column   |
| BASIN  | Constant column | Drop this column  |
| TENDERLOT  | Constant column| Drop this column  |
| SUPPRESSED         | Values are only NaN and 0 |Drop this column |
| JUSTIFICATIONID     | Excessive JOURNEYTIME due to faults  |drop the trip  | 
| ACTUALTIME_ARR              | NaN values present |drop the trip   | 
| ACTUALTIME_DEP  | NaN values present  |drop the trip   | 
| LASTUPDATE| No meaningful FEATURE  |Drop this column | 
| NOTE                                           | No meaningful FEATURE  |Drop this column  | 
| JOURNEYTIME                                           | The presence of less than 0, or extreme values, is removed at 99%.  |drop the trip  | 


## Cleaning data

In [2]:
df = pd.read_csv("/tmp/data/rt_trips_DB_2018.txt",sep=';')

In [3]:
df.head(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,DB,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,DB,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2526331,"


In [4]:
df.shape

(2182637, 16)

In [5]:
df.dtypes

DATASOURCE          object
DAYOFSERVICE        object
TRIPID               int64
LINEID              object
ROUTEID             object
DIRECTION            int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR     float64
ACTUALTIME_DEP     float64
BASIN               object
TENDERLOT          float64
SUPPRESSED         float64
JUSTIFICATIONID    float64
LASTUPDATE          object
NOTE                object
dtype: object

### Check if there are duplicate rows or columns

In [6]:
#remove whitespace in or around feature names
df.columns = df.columns.str.replace(' ', '')

#check to ensure whitespaces have been removed
df.columns

Index(['DATASOURCE', 'DAYOFSERVICE', 'TRIPID', 'LINEID', 'ROUTEID',
       'DIRECTION', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR',
       'ACTUALTIME_DEP', 'BASIN', 'TENDERLOT', 'SUPPRESSED', 'JUSTIFICATIONID',
       'LASTUPDATE', 'NOTE'],
      dtype='object')

In [7]:
#check for duplicate rows

#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', df.duplicated().sum())

# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', df[df.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


In [8]:
# #check if there are any duplicate columns
# # First transpose the df so columns become rows
# dfT = df.T
# print("Number of duplicate (excluding first) columns in the table is: ", dfT.duplicated().sum())
# print("Number of duplicate (including first) columns in the table is: ",  dfT[dfT.duplicated(keep=False)].shape[0])

# #.duplicate returns a Boolean Value so True will be returned if the column is a duplicate
# print("Duplpicated columns will be True: ", dfT.duplicated())

### Check if there is a constant column

In [9]:
#Check the data of category type to see if there is a constant column
df_columns = df.columns
features_card = list(df[df_columns].columns.values)

print('{0:35}  {1}'.format("Feature", "Unique Values"))
print('{0:35}  {1}'.format("-------", "--------------- \n"))

for c in df_columns:
    print('{0:35}  {1}'.format(c, str(len(df[c].unique()))))

Feature                              Unique Values
-------                              --------------- 

DATASOURCE                           1
DAYOFSERVICE                         360
TRIPID                               658964
LINEID                               130
ROUTEID                              588
DIRECTION                            2
PLANNEDTIME_ARR                      64461
PLANNEDTIME_DEP                      791
ACTUALTIME_ARR                       68123
ACTUALTIME_DEP                       66772
BASIN                                1
TENDERLOT                            1
SUPPRESSED                           2
JUSTIFICATIONID                      3527
LASTUPDATE                           360
NOTE                                 46690


The above result shows that DATASOURCE , BASIN , TENDERLOT are constant columns, so delete these three columns

In [10]:
df.drop(labels=['DATASOURCE','BASIN','TENDERLOT'],axis=1,inplace=True)

In [11]:
df.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,NaN,NaN,28-FEB-18 12:05:11,",2526331,"


### Check for missing values

In [12]:
# Prepare %missing column
cols = df.columns
columns_perc_missing  = 100 * (df[cols].isnull().sum()/df.shape[0])
#category_columns_perc_missing

# Store the values in a dataframe
df_perc_missing = pd.DataFrame(columns_perc_missing, columns=['%missing'])
df_perc_missing

,%missing
DAYOFSERVICE,0.000000
TRIPID,0.000000
LINEID,0.000000
ROUTEID,0.000000
DIRECTION,0.000000
PLANNEDTIME_ARR,0.000000
PLANNEDTIME_DEP,0.000000
ACTUALTIME_ARR,6.286295
ACTUALTIME_DEP,7.539091
SUPPRESSED,99.801479


In [13]:
df['SUPPRESSED'].unique()

array([nan,  0.])

The result above shows that there is no SUPPRESSED=1, suppressed trip, so this column can be removed as a constant column

In [14]:
df.drop(labels=['SUPPRESSED'],axis=1,inplace=True)

In [15]:
df.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,JUSTIFICATIONID,LASTUPDATE,NOTE
0,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,NaN,28-FEB-18 12:05:11,",2967409,"
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,NaN,28-FEB-18 12:05:11,",2448968,"
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,NaN,28-FEB-18 12:05:11,",3094242,"
4,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,NaN,28-FEB-18 12:05:11,",2526331,"


In [16]:
df['JUSTIFICATIONID'].unique()

array([    nan, 194642., 205589., ..., 231826., 231813., 231765.])

In [17]:
df[df['JUSTIFICATIONID']==194642]

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,JUSTIFICATIONID,LASTUPDATE,NOTE
1783,18-FEB-18 00:00:00,6269995,45A,45A_60,1,68767,65700,NaN,65746.0,194642.0,26-FEB-18 11:09:33,",2428302,"


In [18]:
df[df['JUSTIFICATIONID']==205589]

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,JUSTIFICATIONID,LASTUPDATE,NOTE
2363,16-MAR-18 00:00:00,6399287,41C,41C_78,1,71543,68400,NaN,68643.0,205589.0,26-MAR-18 12:55:43,",1741586,"


From the above results, we can see that the JUSTIFICATIONID is not equal to 0 in the case of some faulty trips, so these should be removed

In [19]:
df[~df['JUSTIFICATIONID'].isnull()].shape

(4330, 12)

In [20]:
df.drop(df[~df['JUSTIFICATIONID'].isnull()].index,inplace = True)

In [21]:
df.drop(labels=['JUSTIFICATIONID'],axis=1,inplace=True)

In [22]:
df.shape

(2178307, 11)

In [23]:
df.sort_values(by=['LINEID','TRIPID','DAYOFSERVICE'], inplace=True)

In [24]:
df.reset_index(drop=True, inplace=True)

In [25]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE
0,02-JAN-18 00:00:00,5956084,1,1_40,2,39067,36000,38844.0,36104.0,09-JAN-18 20:18:05,",2852798,"
1,03-JAN-18 00:00:00,5956084,1,1_40,2,39067,36000,38900.0,36160.0,11-JAN-18 18:46:25,",2852798,"
2,04-JAN-18 00:00:00,5956084,1,1_40,2,39067,36000,39244.0,36340.0,11-JAN-18 18:55:15,",2852798,"
3,05-JAN-18 00:00:00,5956084,1,1_40,2,39067,36000,39044.0,36519.0,12-JAN-18 20:27:15,",2852798,"
4,02-JAN-18 00:00:00,5956085,1,1_37,1,43837,40800,43681.0,40862.0,09-JAN-18 20:18:05,",2852799,"
...,...,...,...,...,...,...,...,...,...,...,...
2178302,26-DEC-18 00:00:00,8591773,9,9_7,2,67905,63900,67573.0,63908.0,09-JAN-19 17:30:44,",3092424,"
2178303,30-DEC-18 00:00:00,8591773,9,9_7,2,67905,63900,68247.0,63865.0,16-JAN-19 18:16:31,",3092424,"
2178304,23-DEC-18 00:00:00,8591774,9,9_5,1,72908,69600,74306.0,69546.0,08-JAN-19 17:30:40,",3092427,"
2178305,26-DEC-18 00:00:00,8591774,9,9_5,1,72908,69600,73459.0,69576.0,09-JAN-19 17:30:44,",3092427,"


Delete ACTUALTIME_ARR, ACTUALTIME_DEP with empty value

In [26]:
df[df['ACTUALTIME_ARR'].isnull()]

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE
53,01-JAN-18 00:00:00,5958112,1,1_40,2,56378,53400,NaN,53823.0,08-JAN-18 17:21:10,",2856447,"
54,01-JAN-18 00:00:00,5958113,1,1_37,1,59835,57000,NaN,NaN,08-JAN-18 17:21:10,",2856406,"
216,04-JAN-18 00:00:00,5963388,1,1_38,1,29621,28200,NaN,28242.0,11-JAN-18 18:55:15,",2866446,"
318,02-JAN-18 00:00:00,5963731,1,1_41,2,85512,84600,NaN,84619.0,09-JAN-18 20:18:05,",2857795,"
338,02-JAN-18 00:00:00,5965220,1,1_40,2,36781,33600,NaN,33630.0,09-JAN-18 20:18:05,",2857182,"
...,...,...,...,...,...,...,...,...,...,...,...
2178205,28-DEC-18 00:00:00,8586493,9,9_5,1,67361,63000,NaN,62982.0,16-JAN-19 17:31:52,",3104718,"
2178207,31-DEC-18 00:00:00,8586493,9,9_5,1,67361,63000,NaN,63090.0,16-JAN-19 18:27:21,",3104718,"
2178218,24-DEC-18 00:00:00,8587648,9,9_7,2,54381,50400,NaN,50415.0,08-JAN-19 17:53:30,",3099605,"
2178223,24-DEC-18 00:00:00,8587649,9,9_5,1,61053,56700,NaN,56742.0,08-JAN-19 17:53:30,",3099607,"


In [27]:
df.drop(df[df['ACTUALTIME_ARR'].isnull()].index,inplace = True)

In [28]:
df[df['ACTUALTIME_DEP'].isnull()]

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE
51,01-JAN-18 00:00:00,5956268,1,1_40,2,48902,46200,48649.0,NaN,08-JAN-18 17:21:10,",2861592,"
55,01-JAN-18 00:00:00,5958114,1,1_40,2,63498,60600,63044.0,NaN,08-JAN-18 17:21:10,",2856408,"
206,02-JAN-18 00:00:00,5963386,1,1_38,1,25792,24600,25676.0,NaN,09-JAN-18 20:18:05,",1942158,"
207,03-JAN-18 00:00:00,5963386,1,1_38,1,25792,24600,25744.0,NaN,11-JAN-18 18:46:25,",1942158,"
208,04-JAN-18 00:00:00,5963386,1,1_38,1,25792,24600,25695.0,NaN,11-JAN-18 18:55:15,",1942158,"
...,...,...,...,...,...,...,...,...,...,...,...
2178153,28-DEC-18 00:00:00,8586228,9,9_5,1,75635,70800,74647.0,NaN,16-JAN-19 17:31:52,",3105635,"
2178268,24-DEC-18 00:00:00,8589564,9,9_5,1,51097,46800,52316.0,NaN,08-JAN-19 17:53:30,",3105689,"
2178276,31-DEC-18 00:00:00,8589565,9,9_7,2,57081,53100,58005.0,NaN,16-JAN-19 18:27:21,",3105690,"
2178279,28-DEC-18 00:00:00,8589566,9,9_5,1,63753,59400,64761.0,NaN,16-JAN-19 17:31:52,",3105692,"


In [29]:
df.drop(df[df['ACTUALTIME_DEP'].isnull()].index,inplace = True)

In [30]:
df.shape

(1892578, 11)

In [31]:
df[df['ACTUALTIME_DEP'].isnull()]

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE


In [32]:
df[df['ACTUALTIME_ARR'].isnull()]

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE


### Filtering trip files and GTFS files

In [33]:
df_gtfs_routes = pd.read_csv("routes.txt",sep=',')

In [34]:
df_gtfs_routes.head(5)

,route_id,agency_id,route_short_name,route_long_name,route_type
0,10-100-e19-1,01,100,Drogheda Bus Station -,3
1,10-100-e20-1,01,100,Drogheda Bus Station -,3
2,10-101-e19-1,01,101,Busáras - Drogheda Bus Station,3
3,10-101-e20-1,01,101,Busáras - Drogheda Bus Station,3
4,10-103-e19-1,01,103,Dublin (Beresford Place) - Tayto Park,3


In [35]:
df_gtfs_routes.shape

(679, 5)

#### Detection of lines appearing in the no longer existent 2018 gtfs

In [36]:
df_gtfs_routes['route_short_name'].unique()

array(['100', '101', '103', '104', '105', '107', '108', '109', '111',
       '115', '120-1', '120-2', '132', '133', '134', '135', '136', '13',
       '14', '160', '161', '162', '163', '166', '167', '168', '173',
       '174', '175', '182', '187', '190', '201', '202', '203', '205',
       '206', '207', '208', '209', '212', '213', '214', '215', '216',
       '219', '220', '223', '225', '226', '22', '233', '235', '236',
       '237', '239', '23', '240', '241', '243', '245', '248', '252',
       '257', '258', '259', '260', '261', '270', '271', '272', '273',
       '274', '275', '276', '278', '279', '280', '282', '284', '2', '301',
       '302', '303', '304', '305', '306', '30', '313', '314', '320',
       '321', '322', '323', '324', '328', '329', '32', '332', '333',
       '336', '341', '343', '345', '346', '347', '348', '349', '350',
       '355', '360', '362', '365', '366', '370', '371', '372', '373',
       '374', '375', '377', '378', '379', '380', '381', '382', '383',
       '385', '40

Convert letters to upper case to match trip data

In [37]:
df_gtfs_routes['route_short_name']=df_gtfs_routes['route_short_name'].str.upper()

In [38]:
df_gtfs_routes['route_short_name'].unique()

array(['100', '101', '103', '104', '105', '107', '108', '109', '111',
       '115', '120-1', '120-2', '132', '133', '134', '135', '136', '13',
       '14', '160', '161', '162', '163', '166', '167', '168', '173',
       '174', '175', '182', '187', '190', '201', '202', '203', '205',
       '206', '207', '208', '209', '212', '213', '214', '215', '216',
       '219', '220', '223', '225', '226', '22', '233', '235', '236',
       '237', '239', '23', '240', '241', '243', '245', '248', '252',
       '257', '258', '259', '260', '261', '270', '271', '272', '273',
       '274', '275', '276', '278', '279', '280', '282', '284', '2', '301',
       '302', '303', '304', '305', '306', '30', '313', '314', '320',
       '321', '322', '323', '324', '328', '329', '32', '332', '333',
       '336', '341', '343', '345', '346', '347', '348', '349', '350',
       '355', '360', '362', '365', '366', '370', '371', '372', '373',
       '374', '375', '377', '378', '379', '380', '381', '382', '383',
       '385', '40

In [39]:
df['LINEID'].unique()

array(['1', '102', '104', '11', '111', '114', '116', '118', '120', '122',
       '123', '13', '130', '14', '140', '142', '145', '14C', '15', '150',
       '151', '15A', '15B', '15D', '16', '161', '16C', '16D', '17', '17A',
       '18', '184', '185', '220', '236', '238', '239', '25', '25A', '25B',
       '25D', '25X', '26', '27', '270', '27A', '27B', '27X', '29A', '31',
       '31A', '31B', '31D', '32', '32X', '33', '33A', '33B', '33D', '33E',
       '33X', '37', '38', '38A', '38B', '38D', '39', '39A', '39X', '4',
       '40', '40B', '40D', '40E', '41', '41A', '41B', '41C', '41D', '41X',
       '42', '42D', '43', '44', '44B', '45A', '46A', '46E', '47', '49',
       '51D', '51X', '53', '54A', '56A', '59', '61', '63', '65', '65B',
       '66', '66A', '66B', '66X', '67', '67X', '68', '68A', '68X', '69',
       '69X', '7', '70', '70D', '75', '76', '76A', '77A', '77X', '79',
       '79A', '7A', '7B', '7D', '83', '83A', '84', '84A', '84X', '9'],
      dtype=object)

In [40]:
df_gtfs_routes['route_short_name'].nunique()

395

In [41]:
df['LINEID'].nunique()

130

In [42]:
#currentLine为当前的线路
currentLine=[]
currentLine=df_gtfs_routes['route_short_name'].unique()

In [43]:
#dropNoExLineList为要删除的line
dropNoExLineList=[]
for lineid in df["LINEID"].unique():
    if lineid not in currentLine:
        dropNoExLineList.append(lineid)

In [44]:
len(dropNoExLineList)

28

In [45]:
exLineList=[]
for lineid in df["LINEID"].unique():
    if lineid  in currentLine:
        exLineList.append(lineid)

In [46]:
len(exLineList)

102

In [47]:
exLineList

['1',
 '102',
 '104',
 '11',
 '111',
 '114',
 '116',
 '118',
 '120',
 '122',
 '123',
 '13',
 '130',
 '14',
 '140',
 '142',
 '145',
 '15',
 '150',
 '151',
 '15A',
 '15B',
 '15D',
 '16',
 '161',
 '16D',
 '17',
 '18',
 '184',
 '185',
 '220',
 '236',
 '238',
 '239',
 '26',
 '27',
 '270',
 '27A',
 '27B',
 '27X',
 '32',
 '32X',
 '33',
 '33D',
 '33E',
 '33X',
 '37',
 '38',
 '38A',
 '38B',
 '38D',
 '39',
 '39A',
 '39X',
 '4',
 '40',
 '40B',
 '40D',
 '40E',
 '41',
 '41B',
 '41C',
 '41D',
 '41X',
 '42',
 '43',
 '44',
 '44B',
 '46A',
 '46E',
 '47',
 '49',
 '51D',
 '53',
 '54A',
 '56A',
 '59',
 '61',
 '63',
 '65',
 '65B',
 '68',
 '68A',
 '69',
 '69X',
 '7',
 '70',
 '75',
 '76',
 '77A',
 '77X',
 '79',
 '79A',
 '7A',
 '7B',
 '7D',
 '83',
 '83A',
 '84',
 '84A',
 '84X',
 '9']

The results above show that 102 lines are reserved

#### Delete lines that are not available

In [48]:
for dropLine in dropNoExLineList:    
    df.drop(df[df['LINEID']==dropLine].index,inplace = True)

In [49]:
df.shape

(1650494, 11)

In [50]:
df.reset_index(drop=True, inplace=True)

In [51]:
df.drop(labels=['LASTUPDATE','NOTE'],axis=1,inplace=True)

## Combined with weather data

In [52]:
df['DAYOFSERVICE'] = pd.to_datetime(df['DAYOFSERVICE'])

In [53]:
df.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
0,2018-01-02,5956084,1,1_40,2,39067,36000,38844.0,36104.0
1,2018-01-03,5956084,1,1_40,2,39067,36000,38900.0,36160.0
2,2018-01-04,5956084,1,1_40,2,39067,36000,39244.0,36340.0
3,2018-01-05,5956084,1,1_40,2,39067,36000,39044.0,36519.0
4,2018-01-02,5956085,1,1_37,1,43837,40800,43681.0,40862.0


### Add variables month, day, hour and weather for merging according to ACTUALTIME_ARR

In [54]:
df['HOUR'] = (df['ACTUALTIME_ARR']//60)//60

In [55]:
df['MONTH'] = df['DAYOFSERVICE'].dt.month

In [56]:
df['WEEKDAY'] = df['DAYOFSERVICE'].dt.weekday

In [57]:
df_weather = pd.read_csv(f'weatherData_cleaned.csv')

In [58]:
df_weather.shape

(8760, 21)

In [59]:
df_weather.head(5)

,dt,day_hour,timezone,temp,visibility,dew_point,feels_like,temp_min,temp_max,pressure,...,wind_speed,wind_deg,wind_gust,rain_1h,snow_1h,clouds_all,weather_id,weather_main,weather_description,weather_icon
9014,1514764800,2018-01-01 00:00:00,0,4.39,9999.0,2.26,-1.93,4.36,4.56,990,...,12.9,240,18.0,0.0,0.0,40,520,Rain,light intensity shower rain,09n
9015,1514768400,2018-01-01 01:00:00,0,4.39,9999.0,2.26,-1.68,4.36,4.56,990,...,11.8,240,NaN,0.0,0.0,75,520,Rain,light intensity shower rain,09n
9016,1514772000,2018-01-01 02:00:00,0,5.39,9999.0,2.40,-0.47,4.53,5.41,990,...,12.4,240,18.5,0.0,0.0,40,802,Clouds,scattered clouds,03n
9017,1514775600,2018-01-01 03:00:00,0,5.39,9999.0,2.40,-0.47,4.53,5.41,990,...,12.4,240,NaN,0.0,0.0,40,802,Clouds,scattered clouds,03n
9018,1514779200,2018-01-01 04:00:00,0,5.39,9999.0,2.40,-0.33,5.36,5.56,989,...,11.8,240,NaN,0.0,0.0,40,802,Clouds,scattered clouds,03n


### Change columns to upper case consistently

In [60]:
df_weather.columns=df_weather.columns.map(lambda x:x.upper())

In [61]:
dupldate= df_weather[df_weather['DAY_HOUR'].duplicated()]

In [62]:
dupldate

,DT,DAY_HOUR,TIMEZONE,TEMP,VISIBILITY,DEW_POINT,FEELS_LIKE,TEMP_MIN,TEMP_MAX,PRESSURE,...,WIND_SPEED,WIND_DEG,WIND_GUST,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,WEATHER_ICON


In [63]:
df_weather.head(5)

,DT,DAY_HOUR,TIMEZONE,TEMP,VISIBILITY,DEW_POINT,FEELS_LIKE,TEMP_MIN,TEMP_MAX,PRESSURE,...,WIND_SPEED,WIND_DEG,WIND_GUST,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,WEATHER_ICON
9014,1514764800,2018-01-01 00:00:00,0,4.39,9999.0,2.26,-1.93,4.36,4.56,990,...,12.9,240,18.0,0.0,0.0,40,520,Rain,light intensity shower rain,09n
9015,1514768400,2018-01-01 01:00:00,0,4.39,9999.0,2.26,-1.68,4.36,4.56,990,...,11.8,240,NaN,0.0,0.0,75,520,Rain,light intensity shower rain,09n
9016,1514772000,2018-01-01 02:00:00,0,5.39,9999.0,2.40,-0.47,4.53,5.41,990,...,12.4,240,18.5,0.0,0.0,40,802,Clouds,scattered clouds,03n
9017,1514775600,2018-01-01 03:00:00,0,5.39,9999.0,2.40,-0.47,4.53,5.41,990,...,12.4,240,NaN,0.0,0.0,40,802,Clouds,scattered clouds,03n
9018,1514779200,2018-01-01 04:00:00,0,5.39,9999.0,2.40,-0.33,5.36,5.56,989,...,11.8,240,NaN,0.0,0.0,40,802,Clouds,scattered clouds,03n


In [64]:
df.columns=df.columns.map(lambda x:x.upper())

In [65]:
df.head()

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,MONTH,WEEKDAY
0,2018-01-02,5956084,1,1_40,2,39067,36000,38844.0,36104.0,10.0,1,1
1,2018-01-03,5956084,1,1_40,2,39067,36000,38900.0,36160.0,10.0,1,2
2,2018-01-04,5956084,1,1_40,2,39067,36000,39244.0,36340.0,10.0,1,3
3,2018-01-05,5956084,1,1_40,2,39067,36000,39044.0,36519.0,10.0,1,4
4,2018-01-02,5956085,1,1_37,1,43837,40800,43681.0,40862.0,12.0,1,1


### df adds a field DAY_HOUR, unified with weather data

In [66]:
df['DAY_HOUR']=df['DAYOFSERVICE']+pd.to_timedelta(df['HOUR'],unit='H')

In [67]:
df['DAY_HOUR'].dtypes

dtype('<M8[ns]')

In [68]:
df_weather['DAY_HOUR'].dtypes

dtype('O')

In [69]:
df_weather['DAY_HOUR']= pd.to_datetime(df_weather['DAY_HOUR'])

In [70]:
df_weather['DAY_HOUR'].dtypes

dtype('<M8[ns]')

In [71]:
df.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,MONTH,WEEKDAY,DAY_HOUR
0,2018-01-02,5956084,1,1_40,2,39067,36000,38844.0,36104.0,10.0,1,1,2018-01-02 10:00:00
1,2018-01-03,5956084,1,1_40,2,39067,36000,38900.0,36160.0,10.0,1,2,2018-01-03 10:00:00
2,2018-01-04,5956084,1,1_40,2,39067,36000,39244.0,36340.0,10.0,1,3,2018-01-04 10:00:00
3,2018-01-05,5956084,1,1_40,2,39067,36000,39044.0,36519.0,10.0,1,4,2018-01-05 10:00:00
4,2018-01-02,5956085,1,1_37,1,43837,40800,43681.0,40862.0,12.0,1,1,2018-01-02 12:00:00


In [72]:
df.shape

(1650494, 13)

In [73]:
df=pd.merge(df,df_weather,on=['DAY_HOUR'],how='left')
df.drop(columns=['DAY_HOUR'])

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,WIND_SPEED,WIND_DEG,WIND_GUST,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,WEATHER_ICON
0,2018-01-02,5956084,1,1_40,2,39067,36000,38844.0,36104.0,10.0,...,10.80,230,NaN,0.00,0.0,40,802,Clouds,scattered clouds,03d
1,2018-01-03,5956084,1,1_40,2,39067,36000,38900.0,36160.0,10.0,...,12.40,250,NaN,0.00,0.0,20,801,Clouds,few clouds,02d
2,2018-01-04,5956084,1,1_40,2,39067,36000,39244.0,36340.0,10.0,...,7.20,240,NaN,0.16,0.0,75,500,Rain,light rain,10d
3,2018-01-05,5956084,1,1_40,2,39067,36000,39044.0,36519.0,10.0,...,4.60,250,NaN,0.00,0.0,75,803,Clouds,broken clouds,04d
4,2018-01-02,5956085,1,1_37,1,43837,40800,43681.0,40862.0,12.0,...,12.40,250,17.50,0.00,0.0,75,803,Clouds,broken clouds,04d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650489,2018-12-26,8591773,9,9_7,2,67905,63900,67573.0,63908.0,18.0,...,4.60,230,NaN,0.00,0.0,75,803,Clouds,broken clouds,04n
1650490,2018-12-30,8591773,9,9_7,2,67905,63900,68247.0,63865.0,18.0,...,1.34,246,3.58,0.00,0.0,75,300,Drizzle,light intensity drizzle,09n
1650491,2018-12-23,8591774,9,9_5,1,72908,69600,74306.0,69546.0,20.0,...,2.60,360,NaN,0.00,0.0,75,803,Clouds,broken clouds,04n
1650492,2018-12-26,8591774,9,9_5,1,72908,69600,73459.0,69576.0,20.0,...,3.10,210,NaN,0.00,0.0,75,803,Clouds,broken clouds,04n


In [74]:
df.shape

(1650494, 33)

In [75]:
df.sort_values(by=[ 'LINEID','TRIPID', 'DAYOFSERVICE'], inplace=True)

In [76]:
df.drop(labels=['DT','TIMEZONE','DAY_HOUR','TEMP_MIN','TEMP_MAX','PRESSURE','WIND_DEG','WIND_GUST','WEATHER_ICON'],axis=1,inplace=True)

In [77]:
df.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,DEW_POINT,FEELS_LIKE,HUMIDITY,WIND_SPEED,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION
0,2018-01-02,5956084,1,1_40,2,39067,36000,38844.0,36104.0,10.0,...,7.34,5.24,87,10.8,0.00,0.0,40,802,Clouds,scattered clouds
1,2018-01-03,5956084,1,1_40,2,39067,36000,38900.0,36160.0,10.0,...,3.44,2.23,76,12.4,0.00,0.0,20,801,Clouds,few clouds
2,2018-01-04,5956084,1,1_40,2,39067,36000,39244.0,36340.0,10.0,...,7.34,6.10,87,7.2,0.16,0.0,75,500,Rain,light rain
3,2018-01-05,5956084,1,1_40,2,39067,36000,39044.0,36519.0,10.0,...,1.27,-0.47,86,4.6,0.00,0.0,75,803,Clouds,broken clouds
4,2018-01-02,5956085,1,1_37,1,43837,40800,43681.0,40862.0,12.0,...,6.35,9.47,76,12.4,0.00,0.0,75,803,Clouds,broken clouds


## Adding new features

### Add whether the current hour is a rush hour 

In [78]:
rushHourDic={}
for i in range(0, df.shape[0]):
    rushHourDic[i]=0

In [79]:
for i in range(0, df.shape[0]):
    #观察进度
    if i % 10000 == 0:
        print(i)    
    if (df.loc[i, 'HOUR'] in [8,9,10,16,17,18]):
        rushHourDic[i]=1

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [80]:
rushHourDF = pd.DataFrame.from_dict(rushHourDic, orient='index', columns=['RUSHHOUR'])

In [81]:
rushHourDF

,RUSHHOUR
0,1
1,1
2,1
3,1
4,0
...,...
1650489,1
1650490,1
1650491,0
1650492,0


In [82]:
df = df.join(rushHourDF)

In [83]:
df.shape

(1650494, 25)

In [84]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,FEELS_LIKE,HUMIDITY,WIND_SPEED,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,RUSHHOUR
0,2018-01-02,5956084,1,1_40,2,39067,36000,38844.0,36104.0,10.0,...,5.24,87,10.80,0.00,0.0,40,802,Clouds,scattered clouds,1
1,2018-01-03,5956084,1,1_40,2,39067,36000,38900.0,36160.0,10.0,...,2.23,76,12.40,0.00,0.0,20,801,Clouds,few clouds,1
2,2018-01-04,5956084,1,1_40,2,39067,36000,39244.0,36340.0,10.0,...,6.10,87,7.20,0.16,0.0,75,500,Rain,light rain,1
3,2018-01-05,5956084,1,1_40,2,39067,36000,39044.0,36519.0,10.0,...,-0.47,86,4.60,0.00,0.0,75,803,Clouds,broken clouds,1
4,2018-01-02,5956085,1,1_37,1,43837,40800,43681.0,40862.0,12.0,...,9.47,76,12.40,0.00,0.0,75,803,Clouds,broken clouds,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650489,2018-12-26,8591773,9,9_7,2,67905,63900,67573.0,63908.0,18.0,...,7.35,86,4.60,0.00,0.0,75,803,Clouds,broken clouds,1
1650490,2018-12-30,8591773,9,9_7,2,67905,63900,68247.0,63865.0,18.0,...,9.86,85,1.34,0.00,0.0,75,300,Drizzle,light intensity drizzle,1
1650491,2018-12-23,8591774,9,9_5,1,72908,69600,74306.0,69546.0,20.0,...,4.09,94,2.60,0.00,0.0,75,803,Clouds,broken clouds,0
1650492,2018-12-26,8591774,9,9_5,1,72908,69600,73459.0,69576.0,20.0,...,9.63,83,3.10,0.00,0.0,75,803,Clouds,broken clouds,0


### Get new feature JOURNEYTIME

In [85]:
df['JOURNEYTIME']=df['ACTUALTIME_ARR']-df['ACTUALTIME_DEP']

In [86]:
df['PLAN_JOURNEYTIME']=df['PLANNEDTIME_ARR']-df['PLANNEDTIME_DEP']

In [87]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,WIND_SPEED,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,RUSHHOUR,JOURNEYTIME,PLAN_JOURNEYTIME
0,2018-01-02,5956084,1,1_40,2,39067,36000,38844.0,36104.0,10.0,...,10.80,0.00,0.0,40,802,Clouds,scattered clouds,1,2740.0,3067
1,2018-01-03,5956084,1,1_40,2,39067,36000,38900.0,36160.0,10.0,...,12.40,0.00,0.0,20,801,Clouds,few clouds,1,2740.0,3067
2,2018-01-04,5956084,1,1_40,2,39067,36000,39244.0,36340.0,10.0,...,7.20,0.16,0.0,75,500,Rain,light rain,1,2904.0,3067
3,2018-01-05,5956084,1,1_40,2,39067,36000,39044.0,36519.0,10.0,...,4.60,0.00,0.0,75,803,Clouds,broken clouds,1,2525.0,3067
4,2018-01-02,5956085,1,1_37,1,43837,40800,43681.0,40862.0,12.0,...,12.40,0.00,0.0,75,803,Clouds,broken clouds,0,2819.0,3037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650489,2018-12-26,8591773,9,9_7,2,67905,63900,67573.0,63908.0,18.0,...,4.60,0.00,0.0,75,803,Clouds,broken clouds,1,3665.0,4005
1650490,2018-12-30,8591773,9,9_7,2,67905,63900,68247.0,63865.0,18.0,...,1.34,0.00,0.0,75,300,Drizzle,light intensity drizzle,1,4382.0,4005
1650491,2018-12-23,8591774,9,9_5,1,72908,69600,74306.0,69546.0,20.0,...,2.60,0.00,0.0,75,803,Clouds,broken clouds,0,4760.0,3308
1650492,2018-12-26,8591774,9,9_5,1,72908,69600,73459.0,69576.0,20.0,...,3.10,0.00,0.0,75,803,Clouds,broken clouds,0,3883.0,3308


## check for JOURNEYTIME

### Delete cases where journeyTime is negative

In [88]:
joNegdf = df[df['JOURNEYTIME']<0]

In [89]:
joNegdf

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,WIND_SPEED,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,RUSHHOUR,JOURNEYTIME,PLAN_JOURNEYTIME
47815,2018-10-30,8067071,102,102_8,1,39908,36900,40698.0,40738.0,11.0,...,2.6,0.00,0.0,20,801,Clouds,few clouds,0,-40.0,3008
77215,2018-10-27,8064427,11,11_40,1,27703,25200,28773.0,28811.0,7.0,...,6.7,0.00,0.0,40,802,Clouds,scattered clouds,0,-38.0,2503
87446,2018-07-28,7181295,111,111_10,2,84912,84600,84928.0,86089.0,23.0,...,2.1,0.00,0.0,40,802,Clouds,scattered clouds,0,-1161.0,312
111787,2018-06-01,6846732,120,120_7,1,77810,76500,77835.0,79347.0,21.0,...,1.5,0.28,0.0,40,500,Rain,light rain,0,-1512.0,1310
115085,2018-07-06,7148315,120,120_9,2,85653,84600,85923.0,86047.0,23.0,...,1.0,0.00,0.0,75,803,Clouds,broken clouds,0,-124.0,1053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515449,2018-11-23,8128365,79,79_10,1,63594,61200,64034.0,64305.0,17.0,...,4.1,0.00,0.0,75,803,Clouds,broken clouds,1,-271.0,2394
1533206,2018-11-30,8146668,79A,79A_27,1,33521,31500,33615.0,35682.0,9.0,...,5.7,0.00,0.0,20,801,Clouds,few clouds,1,-2067.0,2021
1541408,2018-05-15,6757323,7A,7A_87,2,66360,61200,66721.0,66722.0,18.0,...,7.2,0.19,0.0,75,500,Rain,light rain,1,-1.0,5160
1559693,2018-02-09,6254028,83,83_16,1,38980,33600,38484.0,44625.0,10.0,...,8.2,0.00,0.0,20,801,Clouds,few clouds,1,-6141.0,5380


In [90]:
dropIndex=[]

In [91]:
dropIndex=df[df['JOURNEYTIME']<0].index

In [92]:
df.drop(dropIndex, inplace=True)

In [93]:
joNegdf = df[df['JOURNEYTIME']<0]

In [94]:
joNegdf

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,WIND_SPEED,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,RUSHHOUR,JOURNEYTIME,PLAN_JOURNEYTIME


In [95]:
df.shape

(1650393, 27)

In [96]:
df.reset_index(drop=True, inplace=True)

In [97]:
df.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,WIND_SPEED,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,RUSHHOUR,JOURNEYTIME,PLAN_JOURNEYTIME
0,2018-01-02,5956084,1,1_40,2,39067,36000,38844.0,36104.0,10.0,...,10.8,0.00,0.0,40,802,Clouds,scattered clouds,1,2740.0,3067
1,2018-01-03,5956084,1,1_40,2,39067,36000,38900.0,36160.0,10.0,...,12.4,0.00,0.0,20,801,Clouds,few clouds,1,2740.0,3067
2,2018-01-04,5956084,1,1_40,2,39067,36000,39244.0,36340.0,10.0,...,7.2,0.16,0.0,75,500,Rain,light rain,1,2904.0,3067
3,2018-01-05,5956084,1,1_40,2,39067,36000,39044.0,36519.0,10.0,...,4.6,0.00,0.0,75,803,Clouds,broken clouds,1,2525.0,3067
4,2018-01-02,5956085,1,1_37,1,43837,40800,43681.0,40862.0,12.0,...,12.4,0.00,0.0,75,803,Clouds,broken clouds,0,2819.0,3037


### journeyTime to remove extremes

Because there are some special values in journeyTime, some particularly high or low values, that affect the prediction, the standard deviation method is used to deal with extreme values. 99% of the data lies in the interval: mean - 3SD < x < mean + 3SD , where mean is, defined below as price_mean, and SD is the standard deviation, denoted below as price _std. 99% of the extreme values will be removed after filtering.

In [98]:
topJT=df["JOURNEYTIME"].mean()+3*df["JOURNEYTIME"].std()

In [99]:
topJT

8337.159846185972

In [100]:
bottomJT=df["JOURNEYTIME"].mean()-3*df["JOURNEYTIME"].std()

In [101]:
bottomJT

-499.7292960079235

In [102]:
joTopdf = df[df['JOURNEYTIME']>topJT]

In [103]:
dropIndex=[]

In [104]:
df.shape

(1650393, 27)

In [105]:
dropIndex=df[df['JOURNEYTIME']>topJT].index

In [106]:
joTopdf = df[df['JOURNEYTIME']>topJT]

In [107]:
joTopdf

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,WIND_SPEED,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,RUSHHOUR,JOURNEYTIME,PLAN_JOURNEYTIME
8838,2018-04-13,6626022,1,1_40,2,56002,52800,66256.0,52792.0,18.0,...,2.60,0.00,0.0,75,803,Clouds,broken clouds,1,13464.0,3202
17092,2018-07-15,7169852,1,1_40,2,57151,54600,78209.0,54751.0,21.0,...,2.10,0.11,0.0,75,500,Rain,light rain,0,23458.0,2551
28637,2018-11-22,8125133,1,1_37,1,34903,31500,44181.0,31537.0,12.0,...,5.66,0.00,0.0,40,802,Clouds,scattered clouds,0,12644.0,3403
30220,2018-12-07,8156147,1,1_40,2,79308,76800,79982.0,70690.0,22.0,...,9.30,0.00,0.0,75,500,Rain,light rain,0,9292.0,2508
38077,2018-04-20,6635079,102,102_8,1,55334,51600,56063.0,47570.0,15.0,...,5.10,0.00,0.0,40,802,Clouds,scattered clouds,0,8493.0,3734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631935,2018-07-12,7156100,9,9_5,1,41227,36900,52954.0,36871.0,14.0,...,2.10,0.11,0.0,75,500,Rain,light rain,0,16083.0,4327
1633091,2018-07-25,7175731,9,9_5,1,63142,58500,67222.0,58549.0,18.0,...,4.60,0.00,0.0,75,803,Clouds,broken clouds,1,8673.0,4642
1644170,2018-09-22,8091114,9,9_7,2,52824,48600,57177.0,48602.0,15.0,...,3.10,0.00,0.0,75,803,Clouds,broken clouds,0,8575.0,4224
1648810,2018-12-13,8176588,9,9_7,2,64123,59400,67931.0,59482.0,18.0,...,7.70,0.20,0.0,75,500,Rain,light rain,1,8449.0,4723


In [108]:
df.shape

(1650393, 27)

In [109]:
df.drop(dropIndex, inplace=True)

In [110]:
df.reset_index(drop=True, inplace=True)

Since bottomJT is a negative number, no calculation is necessary

### journeyTime to remove the value equal to 0

In [111]:
joZerodf=df[df['JOURNEYTIME']==0]

In [112]:
joZerodf

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,WIND_SPEED,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,RUSHHOUR,JOURNEYTIME,PLAN_JOURNEYTIME


In [113]:
df.columns

Index(['DAYOFSERVICE', 'TRIPID', 'LINEID', 'ROUTEID', 'DIRECTION',
       'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR',
       'ACTUALTIME_DEP', 'HOUR', 'MONTH', 'WEEKDAY', 'TEMP', 'VISIBILITY',
       'DEW_POINT', 'FEELS_LIKE', 'HUMIDITY', 'WIND_SPEED', 'RAIN_1H',
       'SNOW_1H', 'CLOUDS_ALL', 'WEATHER_ID', 'WEATHER_MAIN',
       'WEATHER_DESCRIPTION', 'RUSHHOUR', 'JOURNEYTIME', 'PLAN_JOURNEYTIME'],
      dtype='object')

In [114]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,WIND_SPEED,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,RUSHHOUR,JOURNEYTIME,PLAN_JOURNEYTIME
0,2018-01-02,5956084,1,1_40,2,39067,36000,38844.0,36104.0,10.0,...,10.80,0.00,0.0,40,802,Clouds,scattered clouds,1,2740.0,3067
1,2018-01-03,5956084,1,1_40,2,39067,36000,38900.0,36160.0,10.0,...,12.40,0.00,0.0,20,801,Clouds,few clouds,1,2740.0,3067
2,2018-01-04,5956084,1,1_40,2,39067,36000,39244.0,36340.0,10.0,...,7.20,0.16,0.0,75,500,Rain,light rain,1,2904.0,3067
3,2018-01-05,5956084,1,1_40,2,39067,36000,39044.0,36519.0,10.0,...,4.60,0.00,0.0,75,803,Clouds,broken clouds,1,2525.0,3067
4,2018-01-02,5956085,1,1_37,1,43837,40800,43681.0,40862.0,12.0,...,12.40,0.00,0.0,75,803,Clouds,broken clouds,0,2819.0,3037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647981,2018-12-26,8591773,9,9_7,2,67905,63900,67573.0,63908.0,18.0,...,4.60,0.00,0.0,75,803,Clouds,broken clouds,1,3665.0,4005
1647982,2018-12-30,8591773,9,9_7,2,67905,63900,68247.0,63865.0,18.0,...,1.34,0.00,0.0,75,300,Drizzle,light intensity drizzle,1,4382.0,4005
1647983,2018-12-23,8591774,9,9_5,1,72908,69600,74306.0,69546.0,20.0,...,2.60,0.00,0.0,75,803,Clouds,broken clouds,0,4760.0,3308
1647984,2018-12-26,8591774,9,9_5,1,72908,69600,73459.0,69576.0,20.0,...,3.10,0.00,0.0,75,803,Clouds,broken clouds,0,3883.0,3308


In [115]:
df['LINEID'].nunique()

102

In [116]:
df['LINEID'].unique()

array(['1', '102', '104', '11', '111', '114', '116', '118', '120', '122',
       '123', '13', '130', '14', '140', '142', '145', '15', '150', '151',
       '15A', '15B', '15D', '16', '161', '16D', '17', '18', '184', '185',
       '220', '236', '238', '239', '26', '27', '270', '27A', '27B', '27X',
       '32', '32X', '33', '33D', '33E', '33X', '37', '38', '38A', '38B',
       '38D', '39', '39A', '39X', '4', '40', '40B', '40D', '40E', '41',
       '41B', '41C', '41D', '41X', '42', '43', '44', '44B', '46A', '46E',
       '47', '49', '51D', '53', '54A', '56A', '59', '61', '63', '65',
       '65B', '68', '68A', '69', '69X', '7', '70', '75', '76', '77A',
       '77X', '79', '79A', '7A', '7B', '7D', '83', '83A', '84', '84A',
       '84X', '9'], dtype=object)

## Combined with PROGRNUMBER_MAX

Get the number of stops on the line corresponding to each trip

In [117]:
df_pro_max = pd.read_csv("lines/PROGRNUMBER_MAX.csv",low_memory=False)

In [118]:
df_pro_max['PROGRNUMBER'].unique()

array([ 76,  75,  73,  68,  41,  42,  38,  64,  59,  47,  78,  71,  62,
        74,  65,  67,  51,  43,  45,  34,  35,  39,  40,   1,  57,  81,
        84,  48,  26,  50,  22,  37,  46,  54,  55,  83,  85,  58,  61,
        56,  53,  52, 102,  97,  63,  90,  86,  29,  70,  69,  49,  23,
        60,  44,  36,  25,  21,  18,  17,  16,  66,  87,  89,  30,  24,
        31,  80,  14,  32,  20,  28,   2,  77,  72,  13,  11,  33,  91,
        19,  94,   8,   9,  10,  27,  15,   6,  95,   4,  79,   5,  88,
       103,   7,  12,   3,  82, 101,  99, 100,  96,  98,  92,  93, 104,
       109])

In [119]:
df_pro_max=df_pro_max.rename(columns={'PROGRNUMBER':'STOP_NUM'})

In [120]:
df_pro_max.head(5)

,TRIPID,DAYOFSERVICE,STOP_NUM
0,5955221,06-JAN-18 00:00:00,76
1,5955222,06-JAN-18 00:00:00,75
2,5955223,06-JAN-18 00:00:00,76
3,5955224,06-JAN-18 00:00:00,75
4,5955225,06-JAN-18 00:00:00,76


In [121]:
df_pro_max.shape

(2182620, 3)

In [122]:
df.shape

(1647986, 27)

In [123]:
df.dtypes

DAYOFSERVICE           datetime64[ns]
TRIPID                          int64
LINEID                         object
ROUTEID                        object
DIRECTION                       int64
PLANNEDTIME_ARR                 int64
PLANNEDTIME_DEP                 int64
ACTUALTIME_ARR                float64
ACTUALTIME_DEP                float64
HOUR                          float64
MONTH                           int64
WEEKDAY                         int64
TEMP                          float64
VISIBILITY                    float64
DEW_POINT                     float64
FEELS_LIKE                    float64
HUMIDITY                        int64
WIND_SPEED                    float64
RAIN_1H                       float64
SNOW_1H                       float64
CLOUDS_ALL                      int64
WEATHER_ID                      int64
WEATHER_MAIN                   object
WEATHER_DESCRIPTION            object
RUSHHOUR                        int64
JOURNEYTIME                   float64
PLAN_JOURNEY

In [124]:
df_pro_max.dtypes

TRIPID           int64
DAYOFSERVICE    object
STOP_NUM         int64
dtype: object

In [125]:
df_pro_max['DAYOFSERVICE'] = pd.to_datetime(df_pro_max['DAYOFSERVICE'])

In [126]:
df=pd.merge(df,df_pro_max,on=['TRIPID','DAYOFSERVICE'],how='left')

In [127]:
df.head(4)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,RAIN_1H,SNOW_1H,CLOUDS_ALL,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,RUSHHOUR,JOURNEYTIME,PLAN_JOURNEYTIME,STOP_NUM
0,2018-01-02,5956084,1,1_40,2,39067,36000,38844.0,36104.0,10.0,...,0.00,0.0,40,802,Clouds,scattered clouds,1,2740.0,3067,43
1,2018-01-03,5956084,1,1_40,2,39067,36000,38900.0,36160.0,10.0,...,0.00,0.0,20,801,Clouds,few clouds,1,2740.0,3067,43
2,2018-01-04,5956084,1,1_40,2,39067,36000,39244.0,36340.0,10.0,...,0.16,0.0,75,500,Rain,light rain,1,2904.0,3067,43
3,2018-01-05,5956084,1,1_40,2,39067,36000,39044.0,36519.0,10.0,...,0.00,0.0,75,803,Clouds,broken clouds,1,2525.0,3067,43


In [128]:
df['LINEID'].nunique()

102

In [129]:
df['LINEID'].unique()

array(['1', '102', '104', '11', '111', '114', '116', '118', '120', '122',
       '123', '13', '130', '14', '140', '142', '145', '15', '150', '151',
       '15A', '15B', '15D', '16', '161', '16D', '17', '18', '184', '185',
       '220', '236', '238', '239', '26', '27', '270', '27A', '27B', '27X',
       '32', '32X', '33', '33D', '33E', '33X', '37', '38', '38A', '38B',
       '38D', '39', '39A', '39X', '4', '40', '40B', '40D', '40E', '41',
       '41B', '41C', '41D', '41X', '42', '43', '44', '44B', '46A', '46E',
       '47', '49', '51D', '53', '54A', '56A', '59', '61', '63', '65',
       '65B', '68', '68A', '69', '69X', '7', '70', '75', '76', '77A',
       '77X', '79', '79A', '7A', '7B', '7D', '83', '83A', '84', '84A',
       '84X', '9'], dtype=object)

In [130]:
df['STOP_NUM'].unique()

array([ 43,  42,  23,  22,  21,  31,  53,  54,  36,  35,  34,  66,  61,
        44,  48,  46,  15,  28,  29,  64,  50,  63,  45,  26,  24,  37,
        52,  30,  47,  17,  90,  87,  91,  88,  33,  89,  86,  32,  55,
        38,  39,  76,  75,  71,  27,  19,  20,  49,  56,  70,  73,  58,
        74,  68,  51,  60,  41,  62,  77,  67,  57,  65,  14,  69,  59,
        18,   7,   8,  13,  16,  84,  80,  81,  25,  40, 102,  97,  95,
       103, 101,  98, 104,  96, 109,  78,  82,  83,  85,  72,  94,  79])

In [131]:
df_pro_all_max=df.groupby(['ROUTEID'], as_index=False)['STOP_NUM'].max()

In [132]:
df_pro_all_max['STOP_NUM'].unique()

array([ 31,  53,  54,  36,  15,  46,  48,  29,  50,  64,  44,  45,  61,
        21,  66,  34,  26,  24,  37,  52,  30,  47,  17,  39,  38,  32,
        90,  91,  33,  87,  88,  89,  55,  86,  27,  19,  49,  56,  20,
        70,  51,  73,  68,  58,  74,  60,  23,  71,  76,  75,  35,  42,
        41,  43,  28,  77,  63,  69,  14,  59,  65,   8,  16,  18,   7,
        13,  62,  22,  84,  80,  81,  25,  40,  57, 109,  98, 103, 104,
        96, 101,  67,  78,  82,  83,  85,  72,  79,  94])

In [133]:
df_pro_all_max=df_pro_all_max.rename(columns={'STOP_NUM':'LINE_STOP_NUM'})

In [134]:
df_pro_all_max.head(5)

,ROUTEID,LINE_STOP_NUM
0,102_10,31
1,102_8,53
2,102_9,54
3,104_15,36
4,104_16,36


In [135]:
df=pd.merge(df,df_pro_all_max,on=['ROUTEID'],how='left')

In [136]:
df1=df[~df['STOP_NUM']==df['LINE_STOP_NUM']]

The result shows that STOP_NUM and LINE_STOP_NUM are equal in number

### Creating target features-AVG_STOP_TIME_MAX

In [137]:
df['AVG_STOP_TIME_MAX']=df['JOURNEYTIME']/df['LINE_STOP_NUM']

In [138]:
df['AVG_STOP_TIME']=df['JOURNEYTIME']/df['STOP_NUM']

From the above results, we can see that AVG_STOP_TIME and AVG_STOP_TIME_MAX have the same value, so AVG_STOP_TIME_MAX is chosen as the target feature

In [139]:
df['AVG_STOP_TIME_MAX']=df['AVG_STOP_TIME_MAX'].astype(int)

In [140]:
df['AVG_STOP_TIME']=df['AVG_STOP_TIME'].astype(int)

In [141]:
df_test=df[df['LINEID']=='185']

In [142]:
df_test.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,HOUR,...,WEATHER_ID,WEATHER_MAIN,WEATHER_DESCRIPTION,RUSHHOUR,JOURNEYTIME,PLAN_JOURNEYTIME,STOP_NUM,LINE_STOP_NUM,AVG_STOP_TIME_MAX,AVG_STOP_TIME
669189,2018-01-06,5959248,185,185_55,1,26673,25200,27632.0,25227.0,7.0,...,500,Rain,light rain,0,2405.0,1473,26,26,92,92
669190,2018-01-06,5959249,185,185_59,2,28777,27600,28815.0,27632.0,8.0,...,803,Clouds,broken clouds,1,1183.0,1177,24,24,49,49
669191,2018-01-06,5959250,185,185_55,1,31640,30000,31582.0,30020.0,8.0,...,803,Clouds,broken clouds,1,1562.0,1640,26,26,60,60
669192,2018-01-06,5959251,185,185_59,2,33755,32400,33672.0,32413.0,9.0,...,500,Rain,light rain,1,1259.0,1355,24,24,52,52
669193,2018-01-06,5959252,185,185_55,1,36440,34800,36567.0,34804.0,10.0,...,803,Clouds,broken clouds,1,1763.0,1640,26,26,67,67


In [143]:
df['LINEID'].nunique()

102

In [144]:
df['LINEID'].unique()

array(['1', '102', '104', '11', '111', '114', '116', '118', '120', '122',
       '123', '13', '130', '14', '140', '142', '145', '15', '150', '151',
       '15A', '15B', '15D', '16', '161', '16D', '17', '18', '184', '185',
       '220', '236', '238', '239', '26', '27', '270', '27A', '27B', '27X',
       '32', '32X', '33', '33D', '33E', '33X', '37', '38', '38A', '38B',
       '38D', '39', '39A', '39X', '4', '40', '40B', '40D', '40E', '41',
       '41B', '41C', '41D', '41X', '42', '43', '44', '44B', '46A', '46E',
       '47', '49', '51D', '53', '54A', '56A', '59', '61', '63', '65',
       '65B', '68', '68A', '69', '69X', '7', '70', '75', '76', '77A',
       '77X', '79', '79A', '7A', '7B', '7D', '83', '83A', '84', '84A',
       '84X', '9'], dtype=object)

In [145]:
df.shape

(1647986, 31)

In [146]:
df['LINEID']=df['LINEID'].str.replace(' ','')

In [147]:
filename = f'allTrips_model.csv'
df.to_csv(filename, index_label=False)